In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import pandas as pd
import joblib
import sys
sys.path.append('../lib')
from utils import edf_loader, combiner, annotations_reader, find_breakpoints, find_sections,\
                    section_seiz, find_edf
from transformers import SpectrogramTransformer
from models import Baseline, PostProcessing
import time

In [4]:
folders1 = find_edf('../dataset/edf/train/01_tcp_ar/')
# folders2 = find_edf('../dataset/edf/train/02_tcp_le/')
# folders3 = find_edf('../dataset/edf/train/03_tcp_ar_a/')
# folders = [folders1, folders2, folders3]

# Test Baseline class

In [5]:
id_folder = 0
folder = folders1[id_folder]
basename = os.path.splitext(os.path.basename(folder))[0]
raw = edf_loader(folder)
raw.info['montage_type'] = 3
tmax = raw.n_times/raw.info['sfreq']
wd = 8
S = SpectrogramTransformer(win_duration=wd)
spectrograms_db_list = S.fit_transform(raw)

In [8]:
%%time
B = Baseline(wd)
B.fit(spectrograms_db_list)
deltaT, predictions = B.predict()

CPU times: user 1.01 s, sys: 30.9 ms, total: 1.04 s
Wall time: 1.04 s


In [15]:
predictions = np.array([1,0,1,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0]).astype(bool)

# Post-Processing

## Remove isolated predictions

In [11]:
minimum_seizure_duration = 15

In [12]:
# Labellization of each event
D = np.diff(np.insert(predictions,0,False))
events_ids = np.cumsum(D)
events_ids = events_ids - min(events_ids)
labels = np.unique(events_ids)

In [13]:
# Compute duration of events
events_sizes,_ = np.histogram(events_ids,bins=len(labels))
events_durations = events_sizes*deltaT

In [14]:
# Correct predictions
new_predictions = predictions[:]
for lab in labels:
    if events_durations[lab] < minimum_seizure_duration:
        new_predictions[events_ids==lab] = False

In [15]:
new_predictions

array([False, False,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False])

## Merge close predictions

In [16]:
minimum_interseizure_gap = 30

In [17]:
# Labellization of each seizure event
D = np.diff(np.insert(new_predictions,0,False))
events_ids = np.cumsum(D)
events_ids = events_ids - min(events_ids)
labels = np.unique(events_ids)

In [18]:
# Compute duration of events
events_sizes,_ = np.histogram(events_ids,bins=len(labels))
events_durations = events_sizes*deltaT

In [19]:
# Correct predictions
for lab in labels[1:-1]:
    if events_durations[lab] < minimum_interseizure_gap:
        new_predictions[events_ids==lab] = True

In [20]:
new_predictions

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False])

# Test lib

In [16]:
P = PostProcessing()
P.fit(deltaT, predictions)

In [17]:
new_pred = P.predict()

In [18]:
new_pred

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False])